# Multi-Factor Model Example

## Introduction and Data

In this notebook, we consider a synthetic example of a multi-factor model and illustrate how the model can be used in a Markowitz mean-variance analysis.

Suppose we can invest in three risky assets whose returns are modelled by a three-factor model

$$R = A + BF + \epsilon,$$

where

 - $\mathsf{E}[F] = \mathbf{0}_3$; standard deviations and correlations of the factors are given in the code below.
 
 - $A = (5\%, 4\%, 6\%)^\top$ and $B = \left[\begin{array}{ccc} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 1 & 0 & 1\end{array}\right]$
 
 - $F$ and $\epsilon$ are jointly Gaussian with $\mathsf{Cov}(F,\epsilon) = \mathbf{0}_{k\times d}$
 
 - $\mathsf{Var}(\epsilon) = \Sigma_\epsilon$ is diagonal, given in the code below.
 
We also assume there is a risk-free rate of return $r_0 = 2\%$.

In [ ]:
# Required libraries
import numpy as np
import scipy as sp

In [ ]:
## GIVEN DATA ON FACTOR MODEL

# Standard deviation of factors (as diagonal matrix)
sd_F = np.array([[0.15, 0, 0], [0, 0.10, 0], [0, 0, 0.10]])

# Correlation matrix of factors
cor_F = np.array([[1, 0, 0.5], [0, 1, 0], [0.5, 0, 1]])

# Covariance matrix of epsilon
cov_eps = np.array([[0.05 ** 2, 0, 0], [0, 0.05 ** 2, 0], [0, 0, 0.05 ** 2]])

# Factor model coefficients
mf_A = np.array([0.05, 0.04, 0.06]).T
mf_B = np.array([[1, 0, 0], [0, 1, 0], [1, 0, 1]])

# Risk-free rate
r0 = 0.02

## Distribution of Returns

Since $F$ and $\epsilon$ are jointly Gaussian, then $R$ has a multivariate normal distribution with mean 

$$M = \mathsf{E}[R] = A + B \mathsf{E}[F] + \mathsf{E}[\epsilon] = A$$

and covariance matrix

$$\Sigma = \mathsf{Var}(A + BF + \epsilon) = B \Sigma_F B^\top + \Sigma_\epsilon.$$

These are also sufficient statistics for the distribution of $R$. These quantities are computed below.

In [ ]:
# Covariance matrix of factors
cov_F = sd_F @ cor_F @ sd_F

# Check if covariance matrix of F is positive definite
np.all(np.linalg.eigvals(cov_F) > 0)    # Check if all eigenvalues are positive; if true, then matrix is p.d.

In [ ]:
# Mean of R
mean_R = mf_A

# Covariance matrix of R
cov_R = mf_B @ cov_F @ mf_B.T + cov_eps

print("Expected returns:", mean_R) 
print("Covariance matrix of returns: \n", cov_R)

In [ ]:
# Check if the covariance matrix of R is positive definite
np.all(np.linalg.eigvals(cov_R) > 0)    # Check if all eigenvalues are positive; if true, then matrix is p.d.

In [ ]:
# Inverse of covariance matrix of returns
inv_cov_R = np.linalg.inv(cov_R)

At this point, we also compute the recurring quantities in the Markowitz analysis, namely $\mathbf{1}_d$, $a := \mathbf{1}_d^\top \Sigma^{-1} \mathbf{1}_d$, and $b := \mathbf{1}_d^\top \Sigma^{-1} M$. 

In [ ]:
# Recurring quantities in Markowitz analysis
vec1 = np.linspace(1, 1, len(mean_R))
a = vec1.T @ inv_cov_R @ vec1
b = vec1.T @ inv_cov_R @ mean_R

In [ ]:
print(vec1, a, b)

We can verify that the risk-free rate $r_0 = 2\%$ abides by the restriction $r_0 < \frac{b}{a}$ in the Markowitz framework:

In [ ]:
# Verify if risk-free rate meets required condition.
r0 < b/a

## Markowitz Analysis

From here, we can calculate the risky asset allocation of the minimum-variance portfolio $\phi_a = \frac{1}{a} \Sigma^{-1} \mathbf{1}_d$ and the tangent portfolio $\phi_\tau = \frac{1}{b - r_0 a} \Sigma^{-1}(M - r_0 \mathbf{1}_d)$.

In [ ]:
# Minimum variance portfolio risky asset allocation
phi_a = (1 / a) * inv_cov_R @ vec1 

# Tangent portfolio risky asset allocation
phi_tau = (1 / (b - r0 * a)) * inv_cov_R @ (mean_R - r0 * vec1)

print(phi_a, phi_tau)

In [ ]:
# Expected return and standard deviation of return of minimum variance portfolio
mean_a = phi_a.T @ mean_R                    # Virtually equal to b/a
sd_a = np.sqrt(phi_a.T @ cov_R @ phi_a)      # Virtually equal to 1/sqrt(a)

print(mean_a, sd_a)

In [ ]:
# Expected return and standard deviation of return of the tangent portfolio
mean_tau = phi_tau.T @ mean_R                
sd_tau = np.sqrt(phi_tau.T @ cov_R @ phi_tau)

# # Virtually equal to theoretical result in Prop 5.2.1. Brugiere (2020):
# mean_tau = r0 + (1 / (b - r0 * a)) * (mean_R - r0 * vec1).T @ inv_cov_R @ (mean_R - r0 * vec1)
# sd_tau = np.sqrt((1 / (b - r0 * a) ** 2) * (mean_R - r0 * vec1).T @ inv_cov_R @ (mean_R - r0 * vec1))

print(mean_tau, sd_tau)

The return of the minimum-variance portfolio and the tangent portfolio can be expressed in terms of the three-factor model by pre-multiplying both sides of the three-factor model by the risky asset allocation of the portfolio under consideration.

For example, the returns of the minimum-variance portfolio can be written in terms of the three-factor model as

$$\mathcal{R}(\phi_a) = \phi_a^\top R = \phi_a^\top (A + BF + \epsilon) = \phi_a^\top A + (\phi_a^\top B) F + \phi_a^\top \epsilon.$$

Since $A = M$, the intercept term yields the expected return $m_{\phi_a}$. The coefficients of the three factors are given by $\phi_a^\top B$,

In [ ]:
phi_a.T @ mf_B

For the tangent portfolio, we have

$$\mathcal{R}(\phi_\tau) = \phi_\tau^\top R = \phi_\tau^\top A + (\phi_\tau^\top B) F + \phi_\tau^\top \epsilon,$$

where $\phi_\tau^\top B$ is computed as

In [ ]:
phi_tau.T @ mf_B

## Beta of Risky Assets from the Factor Model

Using the properties of the covariance and the three-factor model, the covariance between the returns of the risky assets $R$ and the return of the tangent portfolio $\mathcal{R}(\phi_\tau)$ can also be obtained using the three-factor model as follows:

\begin{align*}
\mathsf{Cov}(R, \mathcal{R}(\phi_\tau))
    & = \mathsf{Cov}(R, \phi_\tau^\top R) = \mathsf{Cov}(R, R) (\phi_\tau^\top)^\top = \Sigma \phi_\tau.
\end{align*}

This results to a vector in $\mathbb{R}^3$ whose $i$th entry gives the covariance of the return of the $i$th risky asset and the return of the tangent portfolio.

The beta of the three risky assets with respect to the tangent portfolio, given by the vector $\beta_\tau := (\beta_{\tau,1}, \beta_{\tau,2}, \beta_{\tau, 3})^\top$, is given by 

$$\beta_\tau = \frac{\mathsf{Cov}(R, \mathcal{R}(\phi_\tau))}{\sigma_{\phi_\tau}^2}.$$

In [ ]:
# Beta of risky assets w.r.t. tangent portfolio
beta_risky_assets = cov_R @ phi_tau / (sd_tau ** 2)

print(beta_risky_assets)

We can verify that the security market line holds for the three risky assets if $r_0 + \beta_{\tau_i} (m_{\phi_\tau} - r_0)$ yields $M_i$. 

In [ ]:
# Verify that the security market line holds
r0_vec = np.repeat(r0, len(mean_R))
r0_vec + beta_risky_assets * (mean_tau - r0)

## SML vs. the three-factor model: Error Analysis

**If we are using the security market line as a model,**

$$R_i = r_0 + \beta_{\tau,i}(\mathcal{R}(\phi_\tau) - r_0) + \varepsilon_i,$$

then the variance of the SML error term $\varepsilon_i$ is given by

$$\mathsf{Var}(\varepsilon_i) = \mathsf{Var}(R_i) - \sigma_{\phi_\tau}^2 \beta_{\tau,i}^2, \qquad i=1,\dots,d.$$

(This follows from Proposition 4.3 in Lecture 4 specialized to the portfolio $\varphi_i = (0, \mathbf{e}_i)^\top)$). Since the financial market is fixed, the characteristics of the tangent portfolio and the beta of the $i$th risky asset w.r.t. the tangent portfolio are the same between the SML and the three-factor model. Moreover, we have verified that the SML is valid for the three risky assets, so **we have the same betas under both models**. 

The standard deviation of the SML error terms are computed as follows:

In [ ]:
# Storage for SML error term standard deviations
sd_eps_sml = np.empty(len(mean_R))

# Compute SML error term standard deviations
for i in range(len(mean_R)):
    basis_vec = np.repeat(0, len(mean_R))
    basis_vec[i] = 1
    sd_eps_sml[i] = np.sqrt(basis_vec.T @ cov_R @ basis_vec - (sd_tau ** 2) * (beta_risky_assets[i] ** 2))

In [ ]:
# Display SML error term standard deviations
sd_eps_sml

This calculation shows that the individual error terms in the SML have a higher standard deviation compared to their respective counterparts in the three-factor model. Furthermore, by construction, the error terms in the three-factor model are uncorrelated; a similar statement cannot be conclusively made regarding the SML error terms. **Thus, the three-factor model enables a better explanation of the riskiness of the returns of the assets compared to the explanation/decomposition offerred by the SML, a one-factor model.**

# Arbitrage Pricing Theory (APT) Model Example

Consider a two-factor model $R = A + BF + \epsilon$ consisting of three risky assets, where 

$$\mathsf{E}[R] = \left[\begin{array}{c} 0.05 \\ 0.08 \\ 0.05 \end{array}\right], \qquad B = \left[\begin{array}{cc} 1 & 0 \\ 0 & 1 \\ \frac{1}{3} & \frac{1}{3} \end{array}\right],$$

$\mathsf{E}[\epsilon] = \mathbf{0}_3$, $\mathsf{Cov}(F,\epsilon) = \mathbf{0}_{2\times 3}$, and $\Sigma_F$ invertible.

In [ ]:
# Model inputs
apt_mean_R = np.array([0.05, 0.08, 0.05]).T
apt_B = np.array([[1, 0], [0, 1], [1/3, 1/3]])

## Verifying a Factor Model is an APT Model

According to the theorem discussed in the lecture slides, the given model is an APT model if and only if there exist $\lambda_0 \in \mathbb{R}$ and $\lambda \in \mathbb{R}^2$ such that 

$$\mathsf{E}[R] = \lambda_0 \mathbf{1}_3 + B \lambda.$$

This equation corresponds to the system of three equations and three unknowns

\begin{align*}
0.05 & = \lambda_0 + \lambda_1 \\
0.08 & = \lambda_0 + \lambda_2 \\
0.05 & = \lambda_0 + \frac{1}{3} \lambda_1 + \frac{1}{3} \lambda_2
\end{align*}

In matrix notation, we have

$$\left[\begin{array}{c} 0.05 \\ 0.08 \\ 0.05 \end{array}\right] = \left[\begin{array}{ccc} 1 & 1 & 0 \\ 1 & 0 & 1 \\ 1 & \frac{1}{3} & \frac{1}{3} \end{array}\right] \left[\begin{array}{c} \lambda_0 \\ \lambda_1 \\ \lambda_2 \end{array}\right].$$

The following codes solve the system of equations.

In [ ]:
# Construct coefficient matrix
apt_vec1 = np.array([1] * len(apt_mean_R))
apt_vec1.shape = (3,1)
coef_matrix = np.append(apt_vec1, apt_B, axis = 1)

# Solve the system of equations
apt_lambda = np.linalg.inv(coef_matrix) @ apt_mean_R

print(apt_lambda)

Since a solution exists, we can conclude that the model is an APT model.

## Constructing Risk-Free Investment Portfolios

A risk-free investment portfolio can be constructed by finding $\phi \in \mathbb{R}^3$ such that $\phi^\top B = \mathbf{0}$ with $\phi^\top \mathbf{1}_d \neq 0$; that is, we can choose an element of $\mathsf{Col}(B)$, which is equal to $\mathsf{Null}(B^\top)$. The command ``scipy.linalg.null_space`` gives us an orthonormal basis for the null space of a given matrix.

In [ ]:
# Load null_space command
from scipy.linalg import null_space

# Determine an orthonormal basis for Null(B^T)
basis_nsBt = null_space(apt_B.T)

print(basis_nsBt, sum(basis_nsBt))

Since $\mathsf{Null}(B^\top)$ has only one basis vector (it has dimension 1) and the sum of the components of the basis vector is nonzero, then we can choose $\phi$ equal to the basis vector. Furthermore, the portfolio can be made into an investment portfolio by dividing each entry of $\phi$ by $\phi^\top \mathbf{1}_3$.

In [ ]:
# Construct risk-free investment portfolio
apt_rf_inv_port = basis_nsBt / (basis_nsBt.T @ apt_vec1)

print(apt_rf_inv_port)

Since we have an APT model, then the AAO conditions state that the expected return of this risk-free investment portfolio must be $\lambda_0$. We verify this below.

In [ ]:
# Expected return of the risk-free investment portfolio
apt_rf_inv_port.T @ apt_mean_R